# Problema de Flujo Máximo a Coste Mínimo en un Grafo Dirigido

---

El problema consiste en encontrar el flujo máximo posible entre dos nodos, **s** (fuente) y **t** (sumidero), en un grafo dirigido, sujeto a las capacidades de las aristas y a los costes asociados al flujo en cada arista. El objetivo es maximizar el flujo desde el nodo **s** al nodo **t**, mientras se minimiza el coste total del flujo, que está dado por el coste por unidad de flujo en cada arista.

### Datos:
- Un grafo dirigido con **n** nodos y **m** aristas.
- Cada arista tiene una capacidad máxima de flujo y un coste por unidad de flujo.
- El flujo debe respetar las capacidades de las aristas y debe ser conservado en cada nodo (excepto en los nodos de entrada y salida).
- El objetivo es maximizar el flujo desde el nodo **s** al nodo **t**, minimizando el coste total del flujo.

### Variables de decisión:
- $x_{ij}$ es el flujo en la arista $(i, j)$, donde $i$ y $j$ son nodos del grafo.

### Función objetivo:
Minimizar el coste total del flujo:

$$
\text{Minimizar} \quad \sum_{(i,j) \in A} c_{ij} \cdot x_{ij}
$$

donde $c_{ij}$ es el coste por unidad de flujo en la arista $(i, j)$.

### Restricciones:
1. **Capacidad de flujo**: 

$$
0 \leq x_{ij} \leq u_{ij}
$$

donde $u_{ij}$ es la capacidad de la arista $(i, j)$.

2. **Conservación de flujo**: Para cada nodo $i$ (excepto $s$ y $t$):

$$
\sum_{(j,i) \in A} x_{ji} - \sum_{(i,j) \in A} x_{ij} = 0
$$

El flujo que entra en el nodo $i$ debe ser igual al flujo que sale del nodo $i$.

3. **Flujo en el nodo de origen**: El flujo que sale de **s** es igual al flujo total $F$.

$$
\sum_{(s,j) \in A} x_{sj} = F
$$

4. **Flujo en el nodo de destino**: El flujo que entra en **t** es igual al flujo total $F$.

$$
\sum_{(i,t) \in A} x_{it} = F
$$

---

## Resolución del problema:

A continuación, vamos a resolver este problema utilizando el paquete **JuMP** en **Julia** y el optimizador **GLPK**. Para simplificar, vamos a considerar un grafo con 4 nodos y 5 aristas, con capacidades y costes aleatorios.


In [1]:
# Instalar y cargar los paquetes necesarios
using Pkg
Pkg.add("JuMP")
Pkg.add("GLPK")
using JuMP, GLPK

# Datos del grafo: nodos y aristas
nodos = ["s", "a", "b", "t"]  # Nodos
aristas = [
    ("s", "a", 10, 2),  # (origen, destino, capacidad, coste)
    ("s", "b", 5, 4),
    ("a", "b", 15, 1),
    ("a", "t", 10, 3),
    ("b", "t", 10, 2)
]

# Crear el modelo
model = Model(GLPK.Optimizer)

# Definir las variables de flujo (flujos en las aristas)
@variable(model, f[1:length(aristas)] >= 0)

# Función objetivo: minimizar el coste total del flujo
@objective(model, Min, sum(aristas[i][4] * f[i] for i in 1:length(aristas)))

# Restricciones de capacidad: el flujo en cada arista no puede superar la capacidad
for i in 1:length(aristas)
    @constraint(model, f[i] <= aristas[i][3])
end

# Restricciones de conservación de flujo
# Para cada nodo (excepto s y t), el flujo entrante debe ser igual al flujo saliente
nodos_internos = ["a", "b"]
for nodo in nodos_internos
    # Flujo entrante
    inflow = sum(f[i] for i in 1:length(aristas) if aristas[i][2] == nodo)
    # Flujo saliente
    outflow = sum(f[i] for i in 1:length(aristas) if aristas[i][1] == nodo)
    @constraint(model, inflow - outflow == 0)
end

# Restricción de flujo en el nodo de origen (s)
@constraint(model, sum(f[i] for i in 1:length(aristas) if aristas[i][1] == "s") == 15)

# Restricción de flujo en el nodo de destino (t)
@constraint(model, sum(f[i] for i in 1:length(aristas) if aristas[i][2] == "t") == 15)

# Resolver el modelo
optimize!(model)

# Resultados
println("Estado de la optimización: ", termination_status(model))
println("Flujo óptimo: ", objective_value(model))
for i in 1:length(aristas)
    println("Flujo en la arista ", aristas[i], ": ", value(f[i]))
end


    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`


Estado de la optimización: OPTIMAL
Flujo óptimo: 80.0
Flujo en la arista ("s", "a", 10, 2): 10.0
Flujo en la arista ("s", "b", 5, 4): 5.0
Flujo en la arista ("a", "b", 15, 1): 0.0
Flujo en la arista ("a", "t", 10, 3): 10.0
Flujo en la arista ("b", "t", 10, 2): 5.0
